In [1]:
import pandas as pd 
import numpy as np
import torch 
import pickle
from matplotlib import pyplot as plt

In [2]:
from utils.generate_raw_timeseries import generate_raw_timeseries
from utils.make_tensor import make_tensor

### Generate raw timeseries

In [3]:
%%time 
generate_raw_timeseries(infile='data/in-hospital-mortality/concatenated_X.csv',
                       outfile='data/in-hospital-mortality/raw_hourly_timeseries.pkl')

CPU times: user 17.9 s, sys: 15.3 s, total: 33.2 s
Wall time: 34.7 s


###  Make tensor (mask, discretise, normalise, impute)

In [4]:
%%time
X = pd.read_pickle('data/in-hospital-mortality/raw_hourly_timeseries.pkl')
y = pd.read_csv('data/in-hospital-mortality/concatenated_y.csv')
tensors, tensor_features = make_tensor(X, y, normal_type='mean', impute_type='zero', 
                                 summary_path='data/in-hospital-mortality/summary.pkl')
X_tensor, y_tensor = tensors

/home/kristoforus/miniconda3/envs/deeplearning/lib/python3.8/site-packages/pandas/core/indexing.py:1048: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


CPU times: user 16.9 s, sys: 7.48 s, total: 24.3 s
Wall time: 23.9 s


In [5]:
print(X_tensor.shape)
print(y_tensor.shape)

torch.Size([21139, 48, 46])
torch.Size([21139, 1])


### Split to train, val, test

In [6]:
index = np.arange(X_tensor.shape[0]) 
np.random.shuffle(index)

In [7]:
index_train = index[: int(0.7 * X_tensor.shape[0])]
index_val = index[int(0.7 * X_tensor.shape[0]) : int(0.8* X_tensor.shape[0])] 
index_test = index[int(0.8* X_tensor.shape[0]):]

In [8]:
train = (X_tensor[index_train, :, :], y_tensor[index_train, :])
val = (X_tensor[index_val, :, :], y_tensor[index_val, :])
test = (X_tensor[index_test, :, :], y_tensor[index_test, :])

In [9]:
torch.save((train, val, test), 'data/in-hospital-mortality/tensors.pkl')
torch.save(tensor_features, 'data/in-hospital-mortality/tensor_features.pkl')